# Calculate keepbits for CNAPSv2 daily average files with 99.9% info
snip out a piece of water only and determine keepbits for CNAPS vars using xinfo, using a water only subset

In [1]:
import xarray as xr
import xbitinfo as xb
import fsspec
from pathlib import Path

In [2]:
fs = fsspec.filesystem('file')

In [3]:
flist = fs.glob('/storage/cnapsv2/useast_avg_*.nc')

In [4]:
ds = xr.open_dataset(flist[0], chunks={'ocean_time':1}, drop_variables=['dstart'])

In [5]:
# calculate keepbits on a section with water cells only, and only for first 10 time steps

In [6]:
ds2 = ds[['temp','salt','omega','zeta']].isel(ocean_time=slice(0,10), s_rho=-1, s_w=-4,
               eta_rho=slice(500,700), xi_rho=slice(800,1000))
bitinfo = xb.get_bitinformation(ds2, dim="xi_rho", implementation='python')
keepbits1 = xb.get_keepbits(bitinfo, inflevel=0.999)  # get number of mantissa bits to keep for 99% real information
keepbits1.values

  0%|          | 0/4 [00:00<?, ?it/s]

/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))


<bound method Mapping.values of <xarray.Dataset>
Dimensions:   (inflevel: 1)
Coordinates:
    dim       <U6 'xi_rho'
  * inflevel  (inflevel) float64 0.999
Data variables:
    temp      (inflevel) int64 8
    salt      (inflevel) int64 12
    zeta      (inflevel) int64 5
    omega     (inflevel) int64 1>

In [7]:
ds2 = ds[['v','vbar']].isel(ocean_time=slice(0,100), s_rho=-1,
               eta_v=slice(500,700), xi_v=slice(800,1000))
bitinfo = xb.get_bitinformation(ds2, dim="xi_v", implementation='python')
keepbits2 = xb.get_keepbits(bitinfo, inflevel=0.999)  # get number of mantissa bits to keep for 99% real information
keepbits2.values

  0%|          | 0/2 [00:00<?, ?it/s]

/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))


<bound method Mapping.values of <xarray.Dataset>
Dimensions:   (inflevel: 1)
Coordinates:
    dim       <U4 'xi_v'
  * inflevel  (inflevel) float64 0.999
Data variables:
    v         (inflevel) int64 2
    vbar      (inflevel) int64 2>

In [8]:
ds2 = ds[['u','ubar']].isel(ocean_time=slice(0,100), s_rho=-1,
               eta_u=slice(500,700), xi_u=slice(800,1000))
bitinfo = xb.get_bitinformation(ds2, dim="xi_u", implementation='python')
keepbits3 = xb.get_keepbits(bitinfo, inflevel=0.999)  # get number of mantissa bits to keep for 99% real information
keepbits3.values

  0%|          | 0/2 [00:00<?, ?it/s]

/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))
/home/ec2-user/miniforge3/envs/xbitinfo/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))


<bound method Mapping.values of <xarray.Dataset>
Dimensions:   (inflevel: 1)
Coordinates:
    dim       <U4 'xi_u'
  * inflevel  (inflevel) float64 0.999
Data variables:
    u         (inflevel) int64 3
    ubar      (inflevel) int64 3>

In [9]:
keepbits = xr.merge([keepbits1, keepbits2, keepbits3], compat='override')

In [10]:
keepbits.to_netcdf('keepbits.nc', mode='w')